In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Metabolic_Rate"
cohort = "GSE23025"

# Input paths
in_trait_dir = "../../input/GEO/Metabolic_Rate"
in_cohort_dir = "../../input/GEO/Metabolic_Rate/GSE23025"

# Output paths
out_data_file = "../../output/preprocess/Metabolic_Rate/GSE23025.csv"
out_gene_data_file = "../../output/preprocess/Metabolic_Rate/gene_data/GSE23025.csv"
out_clinical_data_file = "../../output/preprocess/Metabolic_Rate/clinical_data/GSE23025.csv"
json_path = "../../output/preprocess/Metabolic_Rate/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE23025_family.soft.gz', 'GSE23025_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Metabolic_Rate/GSE23025/GSE23025_family.soft.gz
Matrix file: ../../input/GEO/Metabolic_Rate/GSE23025/GSE23025_series_matrix.txt.gz


Background Information:
!Series_title	"Altered Hematopoietic Cell Gene Expression Precedes Development of Therapy-Related Myelodysplasia and Identifies Patients at Risk"
!Series_summary	"Therapy-related myelodysplasia or acute myeloid leukemia (t-MDS/AML) is a lethal complication of cancer treatment. Although t-MDS/AML development is associated with known genotoxic exposures, its pathogenesis is not well understood and methods to predict risk of development of t-MDS/AML in individual cancer survivors are not available. We performed microarray analysis of gene expression in samples from patients who developed t-MDS/AML after autologous hematopoietic cell transplantation (aHCT) for Hodgkin lymphoma (HL) or non-Hodgkin lymphoma (NHL) and controls that did not develop t-MDS/AML after aHCT. CD34+ progenitor cells from peripheral blood stem cell (PBSC) samples obtained pre-aHCT from t-MDS/AML cases and matched controls, and bone marrow (BM) samples obtained at time of development of t-MDS/AM

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
from typing import Optional, Callable, Dict, Any
import json

# 1. Gene Expression Data Availability
# The dataset is focused on microarray analysis of gene expression (HG U133 plus 2.0 Arrays)
is_gene_available = True

# 2.1 Data Availability
# Looking at the sample characteristics dictionary:
# The key 3 contains 'patient status: Case' and 'patient status: Control'
# This can be used as our trait (metabolic rate) as it indicates development of t-MDS/AML
trait_row = 3

# No age information is available in the sample characteristics
age_row = None

# No gender information is available in the sample characteristics  
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert patient status to binary values (0/1)"""
    if value is None:
        return None
    # Extract the value after the colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: Control = 0, Case = 1
    if value.lower() == 'control':
        return 0
    elif value.lower() == 'case':
        return 1
    else:
        return None

def convert_age(value):
    """Convert age to continuous values"""
    # This function is defined but won't be used since age data is not available
    return None

def convert_gender(value):
    """Convert gender to binary values (0/1)"""
    # This function is defined but won't be used since gender data is not available
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # The dictionary provided in the output has keys as rows and lists of values as columns
    sample_chars_dict = {0: ['patient id: 5008', 'patient id: 104', 'patient id: 5035', 'patient id: 138', 'patient id: 5099', 'patient id: 75', 'patient id: 15001', 'patient id: 493', 'patient id: 898', 'patient id: 15002', 'patient id: 445', 'patient id: 15003', 'patient id: 217', 'patient id: 15004', 'patient id: 15', 'patient id: 146', 'patient id: 15005', 'patient id: 696', 'patient id: 15006', 'patient id: 230', 'patient id: 340', 'patient id: 15007', 'patient id: 81', 'patient id: 15008', 'patient id: 80', 'patient id: 15009', 'patient id: 265', 'patient id: 15011', 'patient id: 904', 'patient id: 211'], 
                          1: ['sample type: PBSC', 'sample type: BM'], 
                          2: ['sample set for prediction study: Test', 'sample set for prediction study: Training', 'sample set for prediction study: N/A'], 
                          3: ['patient status: Case', 'patient status: Control'], 
                          4: ['matching group: 5008-Case', 'matching group: 5008-Control', 'matching group: 5035-Case', 'matching group: 5035-Control', 'matching group: 5099-Case', 'matching group: 5099-Control', 'matching group: 15001-Case', 'matching group: 15001-Control', 'matching group: 15002-Case', 'matching group: 15002-Control', 'matching group: 15003-Case', 'matching group: 15003-Control', 'matching group: 15004-Case', 'matching group: 15004-Control', 'matching group: 15005-Case', 'matching group: 15005-Control', 'matching group: 15006-Case', 'matching group: 15006-Control', 'matching group: 15007-Case', 'matching group: 15007-Control', 'matching group: 15008-Case', 'matching group: 15008-Control', 'matching group: 15009-Case', 'matching group: 15009-Control', 'matching group: 15011-Case', 'matching group: 15011-Control', 'matching group: 211-Case', 'matching group: 211-Control', 'matching group: 650-Case', 'matching group: 650-Control']}
    
    # Convert the dictionary to a DataFrame where each row represents a characteristic
    clinical_data = pd.DataFrame.from_dict(sample_chars_dict, orient='index')
    
    # Extract clinical features using the library function
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features_df)
    print("Preview of clinical features:", preview)
    
    # Save the clinical features to a CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features: {0: [1.0], 1: [0.0], 2: [nan], 3: [nan], 4: [nan], 5: [nan], 6: [nan], 7: [nan], 8: [nan], 9: [nan], 10: [nan], 11: [nan], 12: [nan], 13: [nan], 14: [nan], 15: [nan], 16: [nan], 17: [nan], 18: [nan], 19: [nan], 20: [nan], 21: [nan], 22: [nan], 23: [nan], 24: [nan], 25: [nan], 26: [nan], 27: [nan], 28: [nan], 29: [nan]}
Clinical features saved to ../../output/preprocess/Metabolic_Rate/clinical_data/GSE23025.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 54675
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers
# The IDs like '1007_s_at', '1053_at', etc. are Affymetrix probe IDs from microarray data
# These are not human gene symbols and will need to be mapped to official gene symbols
# The "_at" suffix is characteristic of Affymetrix probe identifiers

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns that contain gene identifiers and gene symbols
# From the preview, we can see that 'ID' contains the probe identifiers (like '1007_s_at')
# and 'Gene Symbol' contains the human gene symbols (like 'DDR1 /// MIR4640')
probe_id_column = 'ID'
gene_symbol_column = 'Gene Symbol'

print(f"Using {probe_id_column} column for probe identifiers and {gene_symbol_column} column for gene symbols")

# 2. Get a gene mapping dataframe by extracting these two columns
gene_mapping = get_gene_mapping(gene_annotation, probe_id_column, gene_symbol_column)
print("Gene mapping dataframe created with shape:", gene_mapping.shape)
print("First few rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print("Gene expression data created with shape:", gene_data.shape)
print("First few rows of gene expression data:")
print(gene_data.head())

# 4. Normalize gene symbols to handle synonyms and duplicates
gene_data = normalize_gene_symbols_in_index(gene_data)
print("After normalization, gene expression data has shape:", gene_data.shape)
print("First few normalized gene symbols:")
print(gene_data.index[:10])

# Save the gene expression data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Using ID column for probe identifiers and Gene Symbol column for gene symbols


Gene mapping dataframe created with shape: (45782, 2)
First few rows of gene mapping:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A


Gene expression data created with shape: (21278, 124)
First few rows of gene expression data:
          GSM568139  GSM568140  GSM568141  GSM568142  GSM568143  GSM568144  \
Gene                                                                         
A1BG       5.641500   5.155889   4.670055   5.034909   5.044579   4.691177   
A1BG-AS1   6.013925   5.558621   6.280574   5.596822   5.640726   5.541005   
A1CF       9.308487   7.989697   9.243640   8.018997   7.901563   8.040500   
A2M       12.889525   9.502548  11.594837   9.367717   9.569467   9.504664   
A2M-AS1    7.458487   4.804238   5.134188   5.142312   5.769543   4.882323   

          GSM568145  GSM568146  GSM568147  GSM568148  ...  GSM568253  \
Gene                                                  ...              
A1BG       5.243974   4.745151   4.704808   5.126103  ...   5.494727   
A1BG-AS1   5.945182   5.837597   5.716454   5.313669  ...   3.787006   
A1CF      10.589112   7.931990   7.549126   8.114743  ...   5.014422   

Gene expression data saved to ../../output/preprocess/Metabolic_Rate/gene_data/GSE23025.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data_normalized.shape}")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Re-examine the clinical data from the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print out the sample characteristics to verify available rows
characteristics_dict = get_unique_values_by_row(clinical_data)
print("Sample characteristics dictionary:")
print(characteristics_dict)

# Define conversion functions for the clinical features based on the actual data
def convert_trait(value):
    """Convert patient status to binary (Control=0, Case=1)."""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if 'control' == value:
        return 0.0  # Control
    elif 'case' == value:
        return 1.0  # Case
    else:
        return None  # Unknown status

def convert_age(value):
    """Convert age data to continuous values."""
    # There is no age information in this dataset
    return None

# Create the clinical dataframe using the correct row indices based on sample characteristics
try:
    # Row 3 contains patient status (Case/Control) - this is our trait of interest
    clinical_df = geo_select_clinical_features(
        clinical_data,
        trait="Metabolic_Rate",  # Using this as the trait name as per variable definition
        trait_row=3,  # patient status as the trait
        convert_trait=convert_trait,
        gender_row=None,  # No gender information available
        convert_gender=None,
        age_row=None,     # No age information available
        convert_age=None
    )
    
    print("Clinical data preview:")
    print(preview_df(clinical_df.T))  # Transpose for better viewing
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Saved clinical data to {out_clinical_data_file}")
    
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data_normalized)
    print(f"Shape of linked data: {linked_data.shape}")
    
    # 4. Handle missing values in the linked data
    linked_data_cleaned = handle_missing_values(linked_data, 'Metabolic_Rate')
    print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")
    
    # 5. Check if the trait and demographic features are biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, 'Metabolic_Rate')
    
    # 6. Validate the dataset and save cohort information
    note = "Dataset contains gene expression data from cancer patients who developed t-MDS/AML after autologous hematopoietic cell transplantation. The trait variable is patient status (0=Control, 1=Case)."
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_trait_biased,
        df=unbiased_linked_data,
        note=note
    )
    
    # 7. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Saved processed linked data to {out_data_file}")
    else:
        print("Dataset validation failed. Final linked data not saved.")
        
except Exception as e:
    print(f"Error in processing clinical data: {e}")
    # If we failed to extract clinical data, update the cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=None,
        df=pd.DataFrame(),
        note="Failed to extract clinical data. Gene expression data is available but missing trait information."
    )
    print("Dataset validation failed due to missing clinical data. Only gene data saved.")

Shape of gene data after normalization: (19845, 124)


Saved normalized gene data to ../../output/preprocess/Metabolic_Rate/gene_data/GSE23025.csv


Sample characteristics dictionary:
{0: ['patient id: 5008', 'patient id: 104', 'patient id: 5035', 'patient id: 138', 'patient id: 5099', 'patient id: 75', 'patient id: 15001', 'patient id: 493', 'patient id: 898', 'patient id: 15002', 'patient id: 445', 'patient id: 15003', 'patient id: 217', 'patient id: 15004', 'patient id: 15', 'patient id: 146', 'patient id: 15005', 'patient id: 696', 'patient id: 15006', 'patient id: 230', 'patient id: 340', 'patient id: 15007', 'patient id: 81', 'patient id: 15008', 'patient id: 80', 'patient id: 15009', 'patient id: 265', 'patient id: 15011', 'patient id: 904', 'patient id: 211'], 1: ['sample type: PBSC', 'sample type: BM'], 2: ['sample set for prediction study: Test', 'sample set for prediction study: Training', 'sample set for prediction study: N/A'], 3: ['patient status: Case', 'patient status: Control'], 4: ['matching group: 5008-Case', 'matching group: 5008-Control', 'matching group: 5035-Case', 'matching group: 5035-Control', 'matching gr

Shape of linked data after handling missing values: (124, 19846)
For the feature 'Metabolic_Rate', the least common label is '1.0' with 46 occurrences. This represents 37.10% of the dataset.
The distribution of the feature 'Metabolic_Rate' in this dataset is fine.



Saved processed linked data to ../../output/preprocess/Metabolic_Rate/GSE23025.csv
